In [1]:
import caffe
import numpy as np

In [2]:
caffe.set_mode_cpu()

In [19]:
net = caffe.Net('deploy.prototxt', 'bvlc_reference_caffenet.caffemodel', caffe.TEST)

In [20]:
print(net.blobs['data'].data.shape)

(10, 3, 227, 227)


In [10]:
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_mean('data', np.load('ilsvrc_2012_mean.npy').mean(1).mean(1))
transformer.set_transpose('data', (2,0,1))
transformer.set_channel_swap('data', (2,1,0))
transformer.set_raw_scale('data', 255.0)

In [21]:
#note we can change the batch size on-the-fly
#since we classify only one image, we change batch size from 10 to 1
net.blobs['data'].reshape(1,3,227,227)

im = caffe.io.load_image('cat.jpg')
print(im.shape)
net.blobs['data'].data[...] = transformer.preprocess('data', im)

(360, 480, 3)


In [12]:
out = net.forward()

In [13]:
print(out['prob'].argmax())
labels = np.loadtxt("synset_words.txt", str, delimiter='\t')
top_k = net.blobs['prob'].data[0].flatten().argsort()[-1:-6:-1]
print(labels[top_k])

281
["b'n02123045 tabby, tabby cat'" "b'n02123159 tiger cat'"
 "b'n02124075 Egyptian cat'" "b'n02119022 red fox, Vulpes vulpes'"
 "b'n02127052 lynx, catamount'"]
